In [27]:
import pandas as pd
import numpy as np
import sqlite3
import re
from sklearn.cluster import KMeans

conn = sqlite3.connect("profiles.db")
cursor = conn.cursor()
cursor.execute("ATTACH DATABASE 'colleges.db' AS colleges")

In [28]:
df = pd.read_sql_query("SELECT * FROM colleges.university_profiles AS c JOIN profiles AS p ON c.school = p.school",conn)
max_index = len(df.index)

In [ ]:
def get_type_map(df_column):
    return {t:i for i, t in enumerate(set(df_column))}

def convert_sat_to_act(sat):
    if np.isnan(sat):
        return np.nan
    sat_scores = np.array([990, 1060, 1140, 1210, 1270, 1330, 1390, 1450, 1510, 1560, 1620, 1680, 1740, 1800, 1860, 1920, 1980, 2020, 2080, 2140, 2220, 2290, 2380, 2410])
    return np.where(sat < sat_scores)[0][0] + 13
    
def combine_test_scores(x):
    if not np.isnan(x['act']):
        return x['act']/36.0
    sat = np.array([x['sat_m'], x['sat_r'], x['sat_w']])
    return convert_sat_to_act(3*sat[~np.isnan(sat)].mean())/36.0 if len(sat[np.isnan(sat)]) != 3 else np.nan

def convert_string(x):
    try:
        return int(x)
    except:
        return np.nan

def combine_avg_test_scores(x):
    if not np.isnan(convert_string(x['avg_act'])):
        return convert_string(x['avg_act'])/36.0
    sat = np.array([convert_string(x['sat_math']), convert_string(x['sat_reading']), convert_string(x['sat_writing'])])
    return convert_sat_to_act(3*sat[~np.isnan(sat)].mean())/36.0 if len(sat[np.isnan(sat)]) != 3 else np.nan

def map_statuses(x):
    status_map = {"Denied": 0, "Deferred": np.nan, "Wait-Listed": np.nan, "Accepted": 2, "Will Attend": 2}
    return status_map[x['status']]

def convert_class_rank(x):
    s = str(x['class_rank'])
    if " of " in s:
        return float(s[:s.index(" of ")])/float(s[s.index(" of ") + 4:])
    if "Top " in s:
        return float(s[4:s.index("%")])/100.0
    if "Bottom " in s:
        return float(s[7:s.index("%")])/100.0
    return np.nan

def convert_instate_tuition(x):
    if "<br>" in str(x['cost_attendance']):
        instate = x['cost_attendance'].split("<br>")[0]
        return instate[instate.index("$")+1:]
    else:
        return x['cost_attendance']

def convert_outstate_tuition(x):
    if "<br>" in str(x['cost_attendance']):
        outstate = x['cost_attendance'].split("<br>")[1]
        return outstate[outstate.index("$")+1:]
    else:
        return x['cost_attendance']

def get_tuition(x):
    return x['in_state_tuition'] if x['state'] == x['hs_state'] else x['out_state_tuition']

def map_hs_types(x):
    hs_type_map = {"Public": 0, "Private": 1, "Parochial": 2, "Home": 3}
    return hs_type_map[x['hs_type']]

def map_ins_types(x):
    ins_type_map = {"Public": 0, "Private": 1, "Private for-profit": 1, "&nbsp;": np.nan}
    return ins_type_map[x['institution_type']]

In [ ]:
df = df.loc[:,~df.columns.duplicated()]
df.fillna(value=pd.np.nan, inplace=True)
df['avg_act'] = df['avg_act'].map(lambda x: str(x).strip('-'))
df['test_score'] = df.apply(combine_test_scores, axis=1)
df['avg_test_score'] = df.apply(combine_avg_test_scores, axis=1)
df['class_rank'] = df.apply(convert_class_rank, axis=1)
df['institution_type'] = df.apply(map_ins_types, axis=1)
df['in_state_tuition'] = df.apply(convert_instate_tuition, axis=1)
df['out_state_tuition'] = df.apply(convert_outstate_tuition, axis=1)
#df['tuition'] = df.apply(get_tuition, axis=1)
df['status'] = df.apply(map_statuses, axis=1)

for col in ['cost_attendance','school', 'city', 'sat_m', 'sat_r', 'sat_w', 'act', 'sat_math', 'sat_reading', 'sat_writing', 'avg_act', 'gpa_w']:
    df = df.drop(col, axis=1)
df = df.dropna()#subset=['test_score','avg_test_score','gpa_uw','class_rank','hs_state','hs_type','institution_type'])

df['hs_type'] = df.apply(map_hs_types, axis=1)
replace_columns = ['hs_state', 'state', 'gender']
for col in replace_columns:
    df[col].replace(get_type_map(df[col]), inplace=True)
    
dfc = df.copy()
df_plain = df.copy()
for col in ['avg_gpa','hs_state','hs_type','avg_test_score','state','average_freshman_aid','admission_rate','faculty_total','international_percent','institution_type','female_percentage','year','gender','status','eaed','legacy','athlete','in_state_tuition','out_state_tuition']:
    dfc = dfc.drop(col, axis=1)

def getNextTableIndex(index,j):
    isValid = False
    while(not isValid):
        try:
            dfc['class_rank'][index]
            return index, j+1
        except:
            index += 1

print("Setting up points")
points = []
i = 0
j=0
while i < 97686:
    i, j = getNextTableIndex(i,j)
    point = []
    for col in dfc.columns.values:
        point.append(dfc[col].get(i))
    points.append(point)
    i += 1
print("Done")
        
cluster = KMeans(n_clusters=1000, max_iter=500).fit(points)

print("Assigning clusters")
i = 0
j = 0
while i < 97686:
    label = cluster.labels_[j]
    i, j = getNextTableIndex(i,j)
    for c,col in enumerate(dfc.columns.values):
        dfc.set_value(i, col, cluster.cluster_centers_[label][c])
    i += 1



Setting up points
Done


In [ ]:
for col in ['gpa_uw', 'class_rank', 'test_score']:
    df = df.drop(col, axis=1)

df = df.join(dfc, how='outer')

In [ ]:
from sklearn.model_selection import train_test_split, KFold, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.linear_model import LogisticRegression

k = 1
df0 = df[df.status == 0]
df2 = df[df.status == 2].sample(n = int(k*len(df0)))
df_final = pd.concat([df0, df2])
X = df_final.drop('status', axis=1)
y = df_final['status']

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)


X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y)
mlp = MLPClassifier(hidden_layer_sizes=(8,8,8),max_iter=1000)



# y_pred = cross_val_predict(mlp, X, y, cv = 4)

# print(classification_report(y,y_pred))

# logisticRegr = LogisticRegression()

# y_pred = cross_val_predict(logisticRegr, X, y, cv = 4)

# print(classification_report(y,y_pred))



mlp.fit(X_train1,y_train1)
predictions = mlp.predict(X_test1)
print(confusion_matrix(y_test1,predictions))
print(classification_report(y_test1,predictions))

# logisticRegr = LogisticRegression()
# logisticRegr.fit(X_train1, y_train1)
# score = logisticRegr.score(X_test1, y_test1)
# print("Logistic Regression Score: " + str(score))

In [ ]:
df

# To-do: Jake and Marco
- 